In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time
import json
from collections import namedtuple
import csv

In [2]:
def named_tuple_structure():
    return namedtuple('Player', 'active url name start end pos height weight birthdate college')

In [3]:
def makePlayersDict(verbose=False, nice=True):
    url = 'http://www.basketball-reference.com'
    extension = '/players/'
    soup = BeautifulSoup(requests.get(url+extension).text, "html.parser")
    data = soup.find(id='page_content').p
    links = [url+a['href'] for a in data.find_all('a', href=True)]
    Player = named_tuple_structure()
    d = {}
    for l in links:
        if verbose:
            print l
        if nice:
            time.sleep(1)
        soup = BeautifulSoup(requests.get(l).text, "html.parser")
        data = soup.find(id='players')
        lst = []
        for a in data.find_all('tr'):
            try:
                lst.append([1, url+a.td.strong.a['href']])
            except:
                try:
                    lst.append([0, url+a.td.a['href']])
                except AttributeError:
                    pass
        lst1 = [[a_.get_text() for a_ in a.find_all("td")] for a in data.find_all('tr') if a.find('td') is not None]
        letter_d = {v[0] :Player(*k + v) for k, v in zip(lst, lst1)}
        d.update(letter_d)
    return d       

In [9]:
def dict_to_csv(path = '../data/player_dict.csv'):
    d=makePlayersDict()
    with open(path, 'wb') as f:
        Player = named_tuple_structure()
        w = csv.writer(f)
        w.writerow((['Name']+list(Player._fields)))
        w.writerows(([name] + list(data) for name, data in d.items()))

In [10]:
dict_to_csv()